In [1]:
from scrap import Scraper
from RAG_QnA import RAG_Model
scrp = Scraper()
rag = RAG_Model()

2024-07-30 16:29:05.836747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 16:29:05.860357: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 16:29:05.867944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-30 16:29:07.211930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/junaid-ul-hassan/.cache/huggingface/token
Login successful


In [5]:
url = "https://mistral.ai/news/mistral-large-2407/"

In [6]:
response = scrp.scrape_website(url=url)
if response==403:
    print("This Website don't allow scrapping")

In [7]:
response

200

In [8]:
rag.load_Database()

In [9]:
rag.window_mem.clear()

In [10]:
print(rag.generateResponse("Summerize the context."))

Number of requested results 50 is greater than number of elements in index 21, updating n_results = 21


The text discusses an announcement about the release of Mistral Large 2, a new AI model from Mistral AI. The model boasts improvements over its predecessor in terms of code generation, mathematics, reasoning, multilingual support, and advanced function calling. It has a 128k context window and supports multiple languages and programming languages. It is available on la Plateforme and its weights are hosted on HuggingFace. The company plans to consolidate their offerings around four models: Mistral Nemo, Mistral Large, Codestral, and Embed, while gradually phasing out older ones.</s>


In [7]:
print(rag.window_mem.buffer_as_str)

Human: Summerize the context.
AI: 1. GPT 4o mini is a chatbot using advancements in machine learning and NLP. It offers multimodal support, large context window, enhanced tokenizer, and excels in academic, reasoning, and coding tasks.
2. Its potential uses include serving as a virtual tutor, customer service agent, data analyst, and healthcare assistant.
3. Real-world applications demonstrate its superiority over other small models for tasks like data extraction and email response generation.</s>
